In [1]:
import pickle
import torch
from torch_geometric.data import Data,DataLoader
from functions_refactor import *
from pytorch_util import *
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [4]:
# fixed parameters
hyper_epoch = 50
threshold = -1.3
reuse = False
block = MEGNet_block
head_mol,head_atom,head_edge = head_mol,head_atom,head_edge
epochs = 50
batch_size = 32
clip = 2
lr = 1e-4
epochs_type = epochs

In [ ]:
for _ in range(hyper_epoch):
    # model parameters

    head = np.random.choice([feedforwardHead_Update,feedforwardCombineHead_Update,SimplyInteraction],p=[0.3,0.2,0.5])
    data = np.random.choice(['../Data/{}_data_ACSF_expand_PCA_otherInfo.pickle',\
                             '../Data/{}_data_SOAP_expand_PCA_otherInfo.pickle',\
                             '../Data/{}_data_atomInfo_otherInfo.pickle',\
                             '../Data/{}_data_ACSF_SOAP_atomInfo_otherInfo.pickle'])
    logLoss = np.random.choice([True,False])
    weight = np.random.rand()*2.5 if logLoss else np.random.rand()*0.6
    dim = int(np.random.choice([256,512,768]))
    interleave = np.random.choice([True,False])
    if interleave:
        if dim == 768:
            layer1 = int(np.random.choice([3,4]))
            layer2 = layer1
        else:
            layer1 = int(np.random.choice([3,4,5]))
            layer2 = layer1            
    else:
        if dim == 768:        
            layer1 = int(np.random.choice([3,4]))
            layer2 = int(np.random.choice([3,4]))
        else:
            layer1 = int(np.random.choice([3,4,5]))
            layer2 = int(np.random.choice([3,4,5]))
            
    if dim == 768:
        factor = int(np.random.choice([2,3]))
    else:
        factor = int(np.random.choice([2,3,4]))
        
    aggr = np.random.choice(['mean','add','max'])
    print('\ntraining on {}\n'.format('_'.join([str(i).split('}')[1] if '}' in str(i) else str(i) \
                                        for i in [head,data,dim,logLoss,weight,layer1,layer2,factor,aggr,interleave]])))

    train_dl,val_dl = get_data(data,batch_size)
    model = GNN_multiHead_interleave(reuse,block,head,head_mol,head_atom,head_edge,\
                          dim,layer1,layer2,factor,**data_dict[data],aggr=aggr,interleave=interleave).to('cuda:0')
    paras = trainable_parameter(model)
    opt = Adam(paras,lr=lr)
    scheduler = ReduceLROnPlateau(opt, 'min',factor=0.5,patience=5)
    
    model,train_loss_perType,val_loss_perType,bestWeight = train_type_earlyStop(opt,model,epochs_type,train_dl,val_dl,paras,clip,\
                                                                    scheduler=scheduler,logLoss=logLoss,weight=weight,threshold=threshold)
    if model is not None:
        save_results2(train_loss_perType,val_loss_perType,head,data,dim,logLoss,\
                      weight,layer1,layer2,factor,aggr,interleave)